# NPH CalCS Correlation

After regressing CalCS CO2 flux residuals globally, we noticed a bullseye that looked like the NPH. Perhaps it would be informative to define a box over this bullseye and correlate those residuals with CalCS CO2 flux. 

The interesting thing, is that a composite map of residuals with EOFs of CalCS CO2 flux don't show this signal:

![](https://i.imgur.com/OeSEY43.png)

However, perhaps there is a story that fits. This map shows a correlation generally where the NPH sits in season. When that area has low pressure anomalies, there is outgassing. To me, this indicates that when the NPH moves northward (thus decreasing pressure in that box), we get more advection of cold water into the CCS and stronger upwelling. A composite map tends to show an anomalous High up in the Aleutian area:

![](https://i.imgur.com/vdH18cg.png)

In [36]:
import glob
import numpy as np
import pandas as pd
import xarray as xr
import esmtools as et
import matplotlib.pyplot as plt
%matplotlib inline

#### Load in CCS residuals

In [37]:
filepath = '/glade/p/work/rbrady/EBUS_BGC_Variability/FG_ALT_CO2/CalCS/filtered_output/'
filename = 'calcs-FG_ALT_CO2-residuals-AW-chavez-800km.nc'
CCS = xr.open_dataset(filepath + filename)
CCS = CCS['FG_ALT_CO2_AW']
CCS.name = 'CCS'

#### Load in and filter PSL box

In [38]:
filepath = '/glade/scratch/rbrady/EBUS_BGC_Variability/global_residuals/PSL/remapped/'
PSL = xr.open_mfdataset(filepath + '*.nc', concat_dim='ensemble')
PSL = PSL['PSL']
PSL['time'] = pd.date_range('1920-01', '2016-01', freq='M')
# Convert to hPa
PSL = PSL / 100 

In [39]:
# Filter to box defined above.
PSL = PSL.sel(lat=slice(10, 35), lon=slice(205,240))

# Create area grid
area = 111 * (111 * np.cos(np.deg2rad(PSL.lat)))

# Add columns for longitude.
area = np.expand_dims(area.values, 1)
area = np.repeat(area, PSL.shape[3], axis=1)
PSL['area'] = (('lat','lon'), area)

# Area weight.
PSL = ((PSL * PSL['area'])/PSL['area'].sum()).sum('lat').sum('lon')

/glade/u/home/rbrady/anaconda3/lib/python3.6/site-packages/dask/array/core.py:476: RuntimeWarning: overflow encountered in true_divide
  o = func(*args, **kwargs)


In [60]:
ds = xr.open_dataset('/glade/u/home/rbrady/work/EBUS_BGC_Variability/indices/NPH/NPH.full_ensemble.192001-201512.nc')

In [61]:
ds

<xarray.Dataset>
Dimensions:  (ensemble: 34, time: 1152)
Coordinates:
  * time     (time) datetime64[ns] 1920-01-31 1920-02-29 1920-03-31 ...
Dimensions without coordinates: ensemble
Data variables:
    NPH      (ensemble, time) float64 0.2413 0.2515 -0.09971 -0.6389 -0.4969 ...
Attributes:
    description:  A simple index created over a box spanning 155W to 120W and...

### Correlate

In [31]:
ds = CCS.to_dataset()
ds['PSL'] = PSL

In [32]:
m, r, p = ([] for i in range(3))
for label, group in ds.groupby('ensemble'):
    M, _, R, P, _ = et.stats.linear_regression(group.PSL, group.CCS)
    m.append(M)
    r.append(R)
    p.append(P)
ds = xr.Dataset({'m': ('ensemble', m),
                 'r': ('ensemble', r),
                 'p': ('ensemble', p)})

In [33]:
outpath = '/glade/p/work/rbrady/EBUS_BGC_Variability/area_weighted_regional_regressions/CalCS/FG_ALT_CO2/NPH/'
outfile = 'NPH.FG_ALT_CO2.CalCS.unsmoothed.area_weighted_regional_regression.lag0.nc'
ds.to_netcdf(outpath + outfile)


In [35]:
ds.r

<xarray.DataArray 'r' (ensemble: 34)>
array([-0.253402, -0.253236, -0.260063, -0.191645, -0.177472, -0.261125,
       -0.182097, -0.230003, -0.233664, -0.265484, -0.256838, -0.166479,
       -0.19742 , -0.239522, -0.26593 , -0.221997, -0.265785, -0.254884,
       -0.267387, -0.181852, -0.1691  , -0.241591, -0.212779, -0.210613,
       -0.24984 , -0.171852, -0.191453, -0.250286, -0.219253, -0.278444,
       -0.26201 , -0.251144, -0.185897, -0.229939])
Dimensions without coordinates: ensemble